# Introduction

In this notebook, we will do a comprehensive analysis of the Android app market by comparing thousands of apps in the Google Play store.

# About the Dataset of Google Play Store Apps & Reviews

**Data Source:** <br>
App and review data was scraped from the Google Play Store by Lavanya Gupta in 2018. Original files listed [here](
https://www.kaggle.com/lava18/google-play-store-apps).

# Import Statements

In [6]:
import pandas as pd


# Notebook Presentation

In [7]:
# Show numeric output in decimal format e.g., 2.15
pd.options.display.float_format = '{:,.2f}'.format

In [8]:
# # this for the local version only:
# df_apps = pd.read_csv('apps.csv')
# # alternative, linking to github url of the csv when viewed in "raw" format:
url = 'https://raw.githubusercontent.com/rudil24/pythonCourse/refs/heads/main/solutions/day076/apps.csv' # Replace with your copied URL
df_apps = pd.read_csv(url)

# Read the Dataset

# Data Cleaning

**Challenge**: How many rows and columns does `df_apps` have? What are the column names? Look at a random sample of 5 different rows with [.sample()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html).

In [10]:
df_apps.shape # get the shape of the dataframe in (rows, cols)

(10841, 12)

In [11]:
df_apps.columns # what are the column names of the dataframe?

Index(['App', 'Category', 'Rating', 'Reviews', 'Size_MBs', 'Installs', 'Type',
       'Price', 'Content_Rating', 'Genres', 'Last_Updated', 'Android_Ver'],
      dtype='object')

In [12]:
df_apps.sample(5)

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres,Last_Updated,Android_Ver
8544,Web Browser & Fast Explorer,SOCIAL,4.40,54768,2.60,"5,000,000",Free,0,Everyone,Social,"May 16, 2018",4.0.3 and up
2765,Live Chat - Free Video Talk,DATING,4.70,14,21.00,"5,000",Free,0,Mature 17+,Dating,"August 3, 2018",4.1 and up
9387,MyVodafone (India) - Online Recharge & Pay Bills,PRODUCTIVITY,4.10,1092367,4.00,"10,000,000",Free,0,Everyone,Productivity,"August 4, 2018",Varies with device
9053,Google Allo,COMMUNICATION,4.30,346980,3.50,"10,000,000",Free,0,Everyone,Communication,"January 23, 2018",4.1 and up
411,EC SPORTS,SPORTS,5.00,1,6.30,10,Free,0,Everyone,Sports,"May 26, 2015",2.3.3 and up


### Drop Unused Columns

**Challenge**: Remove the columns called `Last_Updated` and `Android_Version` from the DataFrame. We will not use these columns.

In [13]:
df_apps.drop(['Last_Updated', 'Android_Ver'], axis=1, inplace=True)

### Find and Remove NaN values in Ratings

**Challenge**: How may rows have a NaN value (not-a-number) in the Ratings column? Create DataFrame called `df_apps_clean` that does not include these rows.

In [14]:
print(df_apps['Rating'].isnull().sum())

1474


In [20]:
df_apps_clean = df_apps.dropna(subset=['Rating']).copy()
print(df_apps_clean.shape)

(9367, 10)


### Find and Remove Duplicates

**Challenge**: Are there any duplicates in data? Check for duplicates using the [.duplicated()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html) function. How many entries can you find for the "Instagram" app? Use [.drop_duplicates()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html) to remove any duplicates from `df_apps_clean`.


In [16]:
print(df_apps_clean.duplicated().sum())

476


In [17]:
print(df_apps_clean[df_apps_clean['App'] == 'Instagram'].shape[0])

4


In [23]:
df_apps_clean.drop_duplicates(inplace=True)

# Find Highest Rated Apps

**Challenge**: Identify which apps are the highest rated. What problem might you encounter if you rely exclusively on ratings alone to determine the quality of an app?

In [22]:
df_apps_clean.sort_values(by='Rating', ascending=False).head()

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
186,Basics of Orthopaedics,MEDICAL,5.00,1,5.60,10,Free,0,Everyone,Medical
21,KBA-EZ Health Guide,MEDICAL,5.00,4,25.00,1,Free,0,Everyone,Medical
28,Ra Ga Ba,GAME,5.00,2,20.00,1,Paid,$1.49,Everyone,Arcade
47,Mu.F.O.,GAME,5.00,2,16.00,1,Paid,$0.99,Everyone,Arcade
237,EC Mover,GAME,5.00,5,4.60,10,Free,0,Everyone,Racing


In [24]:
df_apps_clean.sort_values(by=['Rating', 'Reviews'], ascending=[False, False]).head(10)

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
2095,Ríos de Fe,LIFESTYLE,5.00,141,15.00,"1,000",Free,0,Everyone,Lifestyle
2438,"FD Calculator (EMI, SIP, RD & Loan Eligilibility)",FINANCE,5.00,104,2.30,"1,000",Free,0,Everyone,Finance
3115,Oración CX,LIFESTYLE,5.00,103,3.80,"5,000",Free,0,Everyone,Lifestyle
2107,Barisal University App-BU Face,FAMILY,5.00,100,10.00,"1,000",Free,0,Everyone,Education
2069,Master E.K,FAMILY,5.00,90,19.00,"1,000",Free,0,Everyone,Education
1968,CL REPL,TOOLS,5.00,47,17.00,"1,000",Free,0,Everyone,Tools
790,AJ Cam,PHOTOGRAPHY,5.00,44,2.80,100,Free,0,Everyone,Photography
1275,AI Today : Artificial Intelligence News & AI 101,NEWS_AND_MAGAZINES,5.00,43,2.30,100,Free,0,Everyone,News & Magazines
1789,Ek Vote,PRODUCTIVITY,5.00,43,6.20,500,Free,0,Everyone,Productivity
2544,CS & IT Interview Questions,FAMILY,5.00,43,3.30,"1,000",Free,0,Everyone,Education


# Find 5 Largest Apps in terms of Size (MBs)

**Challenge**: What's the size in megabytes (MB) of the largest Android apps in the Google Play Store. Based on the data, do you think there could be limit in place or can developers make apps as large as they please?

# Find the 5 App with Most Reviews

**Challenge**: Which apps have the highest number of reviews? Are there any paid apps among the top 50?

# Plotly Pie and Donut Charts - Visualise Categorical Data: Content Ratings

# Numeric Type Conversion: Examine the Number of Installs

**Challenge**: How many apps had over 1 billion (that's right - BILLION) installations? How many apps just had a single install?

Check the datatype of the Installs column.

Count the number of apps at each level of installations.

Convert the number of installations (the Installs column) to a numeric data type. Hint: this is a 2-step process. You'll have make sure you remove non-numeric characters first.

# Find the Most Expensive Apps, Filter out the Junk, and Calculate a (ballpark) Sales Revenue Estimate

Let's examine the Price column more closely.

**Challenge**: Convert the price column to numeric data. Then investigate the top 20 most expensive apps in the dataset.

Remove all apps that cost more than $250 from the `df_apps_clean` DataFrame.

Add a column called 'Revenue_Estimate' to the DataFrame. This column should hold the price of the app times the number of installs. What are the top 10 highest grossing paid apps according to this estimate? Out of the top 10 highest grossing paid apps, how many are games?


### The most expensive apps sub $250

### Highest Grossing Paid Apps (ballpark estimate)

# Plotly Bar Charts & Scatter Plots: Analysing App Categories

### Vertical Bar Chart - Highest Competition (Number of Apps)

### Horizontal Bar Chart - Most Popular Categories (Highest Downloads)

### Category Concentration - Downloads vs. Competition

**Challenge**:
* First, create a DataFrame that has the number of apps in one column and the number of installs in another:

<img src=https://imgur.com/uQRSlXi.png width="350">

* Then use the [plotly express examples from the documentation](https://plotly.com/python/line-and-scatter/) alongside the [.scatter() API reference](https://plotly.com/python-api-reference/generated/plotly.express.scatter.html)to create scatter plot that looks like this.

<img src=https://imgur.com/cHsqh6a.png>

*Hint*: Use the size, hover_name and color parameters in .scatter(). To scale the yaxis, call .update_layout() and specify that the yaxis should be on a log-scale like so: yaxis=dict(type='log')

# Extracting Nested Data from a Column

**Challenge**: How many different types of genres are there? Can an app belong to more than one genre? Check what happens when you use .value_counts() on a column with nested values? See if you can work around this problem by using the .split() function and the DataFrame's [.stack() method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.stack.html).


# Colour Scales in Plotly Charts - Competition in Genres

**Challenge**: Can you create this chart with the Series containing the genre data?

<img src=https://imgur.com/DbcoQli.png width=400>

Try experimenting with the built in colour scales in Plotly. You can find a full list [here](https://plotly.com/python/builtin-colorscales/).

* Find a way to set the colour scale using the color_continuous_scale parameter.
* Find a way to make the color axis disappear by using coloraxis_showscale.

# Grouped Bar Charts: Free vs. Paid Apps per Category

**Challenge**: Use the plotly express bar [chart examples](https://plotly.com/python/bar-charts/#bar-chart-with-sorted-or-ordered-categories) and the [.bar() API reference](https://plotly.com/python-api-reference/generated/plotly.express.bar.html#plotly.express.bar) to create this bar chart:

<img src=https://imgur.com/LE0XCxA.png>

You'll want to use the `df_free_vs_paid` DataFrame that you created above that has the total number of free and paid apps per category.

See if you can figure out how to get the look above by changing the `categoryorder` to 'total descending' as outlined in the documentation here [here](https://plotly.com/python/categorical-axes/#automatically-sorting-categories-by-name-or-total-value).

# Plotly Box Plots: Lost Downloads for Paid Apps

**Challenge**: Create a box plot that shows the number of Installs for free versus paid apps. How does the median number of installations compare? Is the difference large or small?

Use the [Box Plots Guide](https://plotly.com/python/box-plots/) and the [.box API reference](https://plotly.com/python-api-reference/generated/plotly.express.box.html) to create the following chart.

<img src=https://imgur.com/uVsECT3.png>


# Plotly Box Plots: Revenue by App Category

**Challenge**: See if you can generate the chart below:

<img src=https://imgur.com/v4CiNqX.png>

Looking at the hover text, how much does the median app earn in the Tools category? If developing an Android app costs $30,000 or thereabouts, does the average photography app recoup its development costs?

Hint: I've used 'min ascending' to sort the categories.

# How Much Can You Charge? Examine Paid App Pricing Strategies by Category

**Challenge**: What is the median price price for a paid app? Then compare pricing by category by creating another box plot. But this time examine the prices (instead of the revenue estimates) of the paid apps. I recommend using `{categoryorder':'max descending'}` to sort the categories.